In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

Using TensorFlow backend.


In [0]:
#Kết nối với Drive của bạn
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131352 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [1]:
#Tao thu muc ao link toi GGDrive có tên drive
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
!ls drive # Liet ke cac file trong thu muc để lấy đường dẫn

# import os
# os.getcwd() # Thu muc hien tai
# os.chdir("drive") # Truy cap thu muc drive 
# os.chdir("..") # Quay vee thu muc truoc

basedline_model.py  sample.csv		   sentiment_comment.ipynb  test.crash
glove.6B.50d.txt    sample_submission.csv  submission.csv	    train.crash


In [0]:
class DataSource(object):

    def _load_raw_data(self, filename, is_train=True):

        a = []
        b = []

        regex = 'train_'
        if not is_train:
            regex = 'test_'

        with open(filename, 'r') as file:
            for line in file:
                if regex in line:
                    b.append(a)
                    a = [line]
                elif line != '\n':
                    a.append(line)

        b.append(a)

        return b[1:]

    def _create_row(self, sample, is_train=True):

        d = {}
        d['id'] = sample[0].replace('\n', '')
        review = ""

        if is_train:
            for clause in sample[1:-1]:
                review += clause.replace('\n', '').strip()
            d['label'] = int(sample[-1].replace('\n', ''))
        else:
            for clause in sample[1:]:
                review += clause.replace('\n', '').strip()

        d['review'] = review

        return d

    def load_data(self, filename, is_train=True):

        raw_data = self._load_raw_data(filename, is_train)
        lst = []

        for row in raw_data:
            lst.append(self._create_row(row, is_train))

        return lst


In [0]:
ds = DataSource()
train_data = pd.DataFrame(ds.load_data('drive/DeepLearning_Exs/sentiment_comment/train.crash'))
test_data = pd.DataFrame(ds.load_data('drive/DeepLearning_Exs/sentiment_comment/test.crash', is_train=False))
subm = pd.read_csv('drive/DeepLearning_Exs/sentiment_comment/sample_submission.csv')
list_class = ['0', '1']

In [0]:
train_data.head(20)

,id,label,review
0,train_000000,0,"""Dung dc sp tot cam onshop Đóng gói sản phẩm r..."
1,train_000001,0,""" Chất lượng sản phẩm tuyệt vời . Son mịn nhưn..."
2,train_000002,0,""" Chất lượng sản phẩm tuyệt vời nhưng k có hộp..."
3,train_000003,1,""":(( Mình hơi thất vọng 1 chút vì mình đã kỳ v..."
4,train_000004,1,"""Lần trước mình mua áo gió màu hồng rất ok mà ..."
5,train_000005,0,""" Chất lượng sản phẩm tuyệt vời có điều không ..."
6,train_000006,0,"""Đã nhận đc hàng rất nhanh mới đặt buổi tối mà..."
7,train_000007,1,"""Các siêu phẩm thấy cấu hình toàn tựa tựa nhau..."
8,train_000008,0,"""Hàng ship nhanh chất lượng tốt tư vấn nhiệt..."
9,train_000009,1,"""Đồng hồ đẹp nhưng 1 cái đứt dây 1 cái k chạy..."


In [0]:
test_data.head()

,id,review
0,test_000000,"""Chưa dùng thử nên chưa biết"""
1,test_000001,""" Không đáng tiềnVì ngay đợt sale nên mới mua ..."
2,test_000002,"""Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắ..."
3,test_000003,"""Vải đẹp.phom oki luôn.quá ưng"""
4,test_000004,"""Chuẩn hàng đóng gói đẹp"""


In [0]:
train_data['review'][3]

'":(( M\xc3\xacnh h\xc6\xa1i th\xe1\xba\xa5t v\xe1\xbb\x8dng 1 ch\xc3\xbat v\xc3\xac m\xc3\xacnh \xc4\x91\xc3\xa3 k\xe1\xbb\xb3 v\xe1\xbb\x8dng cu\xe1\xbb\x91n s\xc3\xa1ch kh\xc3\xa1 nhi\xe1\xbb\x81u hi v\xe1\xbb\x8dng n\xc3\xb3 s\xe1\xba\xbd n\xc3\xb3i v\xe1\xbb\x81 vi\xe1\xbb\x87c h\xe1\xbb\x8dc t\xe1\xba\xadp c\xe1\xbb\xa7a c\xc3\xa1ch sinh vi\xc3\xaan tr\xc6\xb0\xe1\xbb\x9dng Harvard ra sao nh\xe1\xbb\xafng n\xe1\xbb\x97 l\xe1\xbb\xb1c c\xe1\xbb\xa7a h\xe1\xbb\x8d nh\xc6\xb0 th\xe1\xba\xbf n\xc3\xa0o 4h s\xc3\xa1ng? t\xe1\xba\xa1i sao h\xe1\xbb\x8d l\xe1\xba\xa1i ph\xe1\xba\xa3i th\xe1\xbb\xa9c d\xe1\xba\xady v\xc3\xa0o th\xe1\xbb\x9di kh\xe1\xba\xafc \xc4\x91\xe1\xba\xa5y? sau \xc4\x91\xc3\xb3 l\xc3\xa0 c\xe1\xba\xa3 m\xe1\xbb\x99t c\xc3\xa2u chuy\xe1\xbb\x87n ra sao. C\xc3\xa1i m\xc3\xacnh th\xe1\xbb\xb1c s\xe1\xbb\xb1 c\xe1\xba\xa7n \xe1\xbb\x9f \xc4\x91\xc3\xa2y l\xc3\xa0 c\xc3\xa2u chuy\xe1\xbb\x87n \xe1\xba\xa9n d\xe1\xba\xa5u trong \xc4\x91\xc3\xb3 \xc4\x91\xe1\xbb\x83 t\xe1

In [0]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use
EMBEDDING_FILE = 'drive/DeepLearning_Exs/sentiment_comment/glove.6B.50d.txt'

In [0]:
#Tiền xử lý dữ liệu comment 

def nomarlize(row_data):
  #Bỏ hết các dấu câu thừa
  #Làm sạch data
  #Đổi các từ viết tắt thành từ đầy đủ
  #Lấy được feature từ các emojis
  
  return row_data


In [0]:
tokenizer = Tokenizer(num_words=max_features)
train_data = nomarlize(list(train_data.review))
test_data = nomarlize(list(test_data.review))

tokenizer.fit_on_texts(train_data))
list_tokenized_train = tokenizer.texts_to_sequences(train_data)
list_tokenized_test = tokenizer.texts_to_sequences(test_data)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)


In [0]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940464, 0.6441042)

In [0]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [0]:
embedding_matrix.shape

(12120, 50)

In [0]:
embed_size

50

In [0]:
inp = Input(shape=(maxlen,))
x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],  weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
y = train_data.label


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 50)           606000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 100)          40400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total para

In [0]:
file_path="drive/DeepLearning_Exs/sentiment_comment/weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)


callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=32, epochs=5, validation_split=0.05, callbacks=callbacks_list)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 14478 samples, validate on 1609 samples
Epoch 1/5
14478/14478 [==============================] - 244s 17ms/step - loss: 0.3906 - acc: 0.8121 - val_loss: 0.3130 - val_acc: 0.8676

Epoch 00001: val_loss improved from inf to 0.31304, saving model to weights_base.best.hdf5
Epoch 2/5
14478/14478 [==============================] - 244s 17ms/step - loss: 0.2610 - acc: 0.8920 - val_loss: 0.2816 - val_acc: 0.8807

Epoch 00002: val_loss improved from 0.31304 to 0.28163, saving model to weights_base.best.hdf5
Epoch 3/5
14478/14478 [==============================] - 242s 17ms/step - loss: 0.2248 - acc: 0.9097 - val_loss: 0.2738 - val_acc: 0.8850

Epoch 00003: val_loss improved from 0.28163 to 0.27376, saving model to weights_base.best.hdf5
Epoch 4/5
14478/14478 [==============================] - 242s 17ms/step - loss: 0.1954 - acc: 0.9243 - val_loss: 0.2786 - val_a

In [0]:
model.load_weights(file_path)

y_test = model.predict([X_te], batch_size=1024, verbose=1)
submitssion = []
for y_t in y_test:
  submitssion.append(int(round(y_t,0)))
test_data['label'] = submitssion
test_data[['id', 'label']].to_csv('drive/DeepLearning_Exs/sentiment_comment/submission.csv', index=False)

10981/10981 [==============================] - 2s 201us/step
